In [ ]:
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from scipy.misc import face, imresize
from sklearn.feature_extraction.image import extract_patches_2d
from sklearn.feature_extraction.image import reconstruct_from_patches_2d
from sklearn.decomposition import MiniBatchDictionaryLearning

In [ ]:
# Step 1: Load the image
try:  # SciPy >= 0.16 have face in misc
    from scipy.misc import face
    face = face(gray=True)
except ImportError:
    face = sp.face(gray=True)
    
print(face.shape)
plt.imshow(face, cmap='gray')
plt.show() 
H, W = face.shape
pH, pW = 4,4
face = imresize(face, (H/2,W/2))
print(face.shape)

In [ ]:
# Step 2: Sample the patches from the image to learn. 
# Let set the number of patches to 30000
patches = ### YOUR CODE HERE ###
print('Number of patches: %d' %patches.shape[0],)

# randomly shuffle the patches
### YOUR CODE HERE ###

print('Visualization of some patches')
for i in range(10):
    for j in range(10):
        plt.subplot(10,10,i*10+j+1)
        plt.imshow(patches[i*10+j,...], cmap='gray')
plt.show()

In [ ]:
# Step 3: Normalize the data to train the dictionary

# create matrix X as a reshape of the patches 
# X has the shape (num_patches x num_features)
### YOUR CODE HERE ###

# convert X to float type (np.float64)
### YOUR CODE HERE ###

# Subtract mean and divide X by its standard deviation
### YOUR CODE HERE ###

In [ ]:
# Step 4: Learn the dictionary from the patches
# create MiniBatchDictionaryLearning object 
# with: 64 components, alpha = 1, 500 iterations,
# using 'lars' algorithm and use sparse coding with 6 non-zero coefficients

d_learner = ### YOUR CODE HERE ###
# learn the dictionary
### YOUR CODE HERE ###
# get the dictionary
D = d_learner.components_
print(D.shape)

In [ ]:
# Utility functions
def sample_patches(im, pH, pW):
    H, W = im.shape
    patches = list([])
    n = H / pH
    m = W / pW
    for i in range(n):
        for j in range(m):
            patch = im[i * pH:(i+1) * pH, j * pW : (j+1) * pW]
            patches.append(patch.reshape([1,-1]))
    patches = np.vstack(patches)
    return patches

def patches_to_img(patches, pH, pW, H, W):
    im = np.zeros((H,W))
    n = H / pH
    m = W / pW
    for i in range(n):
        for j in range(m):
            im[i * pH:(i+1) * pH, j * pW : (j+1) * pW] = patches[i*m+j]
    return im

In [ ]:
# Step 5: prepare patches to run the reconstruction
patches = (face, pH, pW)
patches = patches.astype(np.float64)
patches = patches.reshape([patches.shape[0],-1])
mean = patches.mean(axis=0)
patches -= mean

In [ ]:
# Step 6: Reconstruct the image with different number of non-zero coefficients
transform_algorithms = [
     {'transform_n_nonzero_coefs': 1},
     {'transform_n_nonzero_coefs': 2},
     {'transform_n_nonzero_coefs': 3},
     {'transform_n_nonzero_coefs': 4},
     {'transform_n_nonzero_coefs': 5},
     {'transform_n_nonzero_coefs': 6},
]
for kwargs in transform_algorithms:
    d_learner.set_params(transform_algorithm='lars', **kwargs)
    # find the sparse representaion fo the patches
    sparse_reps = ### YOUR CODE HERE ###
    
    # reconstruct the patches from their sparse representations and the learned dictionary
    recon_patches = ### YOUR CODE HERE ###
    
    # add the mean to the reconstruction
    recon_patches = ### YOUR CODE HERE ###
    
    # reshape recon_patches to num_patches x pH x pW
    recon_patches = ### YOUR CODE HERE ###
    
    # put all the patches back into the reconstructed image
    recon = patches_to_img(recon_patches, pH, pW, H/2, W/2)
    
    plt.title('Num Coefs %d - Reduction rate %.02f' %(kwargs['transform_n_nonzero_coefs'], 
                                                   (1.0 - float(kwargs['transform_n_nonzero_coefs']) / (pH * pW)) ))
    plt.show()    